In [2]:
import sys
username = 'admin' # 'admin' for BO1 lab
sys.path.append('c:/users/'+ username +'/miniconda3/lib/site-packages')
import qontrol
import os

# file name
name = "First_test.txt" #"Resistance_measurements_Test_Q5-04-10_CHN0_T25T16_MZI.txt"
path = "G:/Shared drives/FMPS Archive/Projects/0_QPIXPAD/QPX4/Opt.Meas/20211013_Q3_+02_+08/Q8b" #"G:/Shared drives/FMPS Archive/Projects/0_QPIXPAD/QPX4/Opt.Meas/20210728_Q5_-04_-10_nodp/Q8b/"
filename = path + name
# Setup Qontroller
serial_port_name = "COM3" #"COM3" #"/dev/tty.usbserial-FT06QAZ5" # name of the USB Serial Port
q = qontrol.QXOutput(serial_port_name = serial_port_name, response_timeout = 0.1)

print ("Qontroller '{:}' initialised with firmware {:} and {:} channels".format(q.device_id, q.firmware, q.n_chs) )

# Set voltage on each channel to its index in volts, read voltage, current

channel_list = [k for k in range(8)] # the channels to retrieve resistance measurements from
voltage_list = [3.0 for k in range(8)] # the voltages to be applied to each channel

measured_voltage_list = [] # the voltages read 
measured_current_list = [] # the current read 
derived_resistance_list = [] # the derived resistances

max_current = 16 # current compliance #16
max_voltage = 3 # voltage compliance

# resolutions for computing measurement error

current_resolution = 0.049 # in mA =  49 uA
voltage_resolution = 0.000180 # in V = 180 uV

#Set current for channel K
#I = 0
#q.i[K] = I

# Set the compliances
for channel in range(q.n_chs):
    q.imax[channel] = max_current
    q.vmax[channel] = max_voltage
    
#Set voltage for each channel
for channel in range(len(channel_list)):
    q.v[channel_list[channel]] = voltage_list[channel] 
    
# Open the log file
meas_file = open(filename, "a")
if os.stat(filename).st_size == 0:
    meas_file.write( "Channel,Voltage[V],Measured_Voltage[V],Measured_Current[mA],Derived_Resistance[Ohm]\n")

Qontroller 'Q8b-0471' initialised with firmware v2.4.1 and 32 channels


In [ ]:
# perform a single measurement from all channels, print them on screen and log them on file
for channel in range(len(channel_list)):
    measured_voltage_list.append(q.v[channel_list[channel]])
    measured_current_list.append(q.i[channel_list[channel]])
    if (measured_current_list[-1]>=current_resolution):
        derived_resistance_list.append((measured_voltage_list[-1]/measured_current_list[-1])*1000)
    else:
         derived_resistance_list.append(-1)
    print("Channel {:} set to {:} V, measured {:} V and {:+05.2f} mA\n".format(channel_list[channel], voltage_list[channel], measured_voltage_list[-1], measured_current_list[-1]))
    print("The derived resistance is: {:} Ohm\n".format(derived_resistance_list[-1]))
    meas_file.write(str(channel_list[channel]) + "," + str(voltage_list[channel]) + "," + str(measured_voltage_list[-1]) + "," + str(measured_current_list[-1]) + "," + str(derived_resistance_list[-1])+ "\n")

# Close the log file
meas_file.close()

# Set all channels to zero
q.v[:] = 0
q.i[:] = 0

q.close()

#help(qontrol.QXOutput)